TODO : 
 - Colocar o relogio a atualizar periodicament (no main)
 - Criar dois relógios e verigficar se os ts são iguais
 - Guardar historico de offsets
 - Guardar historico de rates
 - Ver se as timeslote coincidem


In [8]:
import socket
import struct
import time
from datetime import datetime, timedelta
import threading
import sys

In [9]:
class AbstractClock():
    def __init__(self):

        # Clock correction parameters
        self.delay = 0
        self.last_offset = 0
        self.offset = 0
        self.rate = 1
        self.last_rate = 1
        
        # Time parameters for rate
        self.start_datetime = datetime.now() # datetime of when the clock was started
        self.start_monotonic = time.monotonic() # monotonic time of when the clock was started

        # NTP client
        self.ntp_client = NTPclient(self.start_monotonic, self.start_datetime)
        self.ntp_timestamp = self.ntp_client.getServerTime()[2] # datetime of when the NTP server was polled
        self.last_ntp_timestamp = self.ntp_timestamp

        # intrinsic clock attributes
        self.timestamp = time.monotonic()
        self.last_timestamp = self.timestamp

    def correctClock(self):
        ntp_time = self.ntp_client.getServerTime()
        t0 = ntp_time[1]
        t1 = ntp_time[2]
        t2 = ntp_time[3]
        t3 = ntp_time[4]
        # Update the rate parameters
        self.last_ntp_timestamp = self.ntp_timestamp
        self.ntp_timestamp = t1
        self.last_timestamp = self.timestamp
        self.timestamp = time.monotonic()

        self.updateOffset(t0, t1, t2, t3)
        self.updateRate(t0, t1, t2, t3)
        self.updateDelay(t0, t1, t2, t3)


    def updateOffset(self, t0, t1, t2, t3):
        self.last_offset = self.offset
        self.offset = ( t1-t3 + t2-t0) / 2

    def updateRate(self, t0, t1, t2, t3):
        self.last_rate = self.rate
        self.rate = abs((self.timestamp - self.last_timestamp) / (self.ntp_timestamp - self.last_ntp_timestamp).total_seconds())

    def updateDelay(self, t0, t1, t2, t3):
        self.delay = (t3-t0) - (t2-t1)

    def getCorrectedTime(self):
        now = time.monotonic()
        elapsed_time = now - self.timestamp
        
        corrected_time = self.timestamp + elapsed_time*self.rate + self.offset.total_seconds()


In [10]:
class NTPclient():
    def __init__(self, start_monotonic, start_datetime):
        # Connection parameters
        self.host = '0.pool.ntp.org' # case of NTP online server
        self.port = 123
        self.read_buffer = 1024
        self.address = (self.host, self.port)
        self.epoch = 2208988800
        self.client = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)

        self.start_monotonic = start_monotonic
        self.start_datetime = start_datetime

    def monotonicToDatetime(self, now, start = 0):
        variation = timedelta(seconds=(now-start))
        elapsed_datetime = self.start_datetime + variation

        return elapsed_datetime


    def getServerTime(self):

        '''
        This function communicates with an NTP server to get the server time. It sends a request to the server, 
        receives the response, unpacks the timestamps from the response, and converts these timestamps to datetime objects.

        The function returns five datetime objects:
        - ref_time: Reference since the NTP clock started (Jan. 1st, 1900)
        - orig_time: The local time when the request was sent
        - rx_time: The time at the NTP server when the response was received.
        - tx_time: The time at the NTP server when the response was sent.
        - dest_time: The local time when the response was received.
        '''

        data = b'\x1b' + 47 * b'\0'
        orig_time = time.monotonic()
        self.client.sendto(data, self.address)
        data, self.address = self.client.recvfrom(self.read_buffer)
        orig_int, orig_frac, ref_int, ref_frac, rx_int, rx_frac, tx_int, tx_frac = struct.unpack("!12I", data)[4:12]


        rx_int -= self.epoch
        tx_int -= self.epoch

        # Convert the fractional part to microseconds (1 second = 2**32 fractional units)
        ref_frac = ref_frac * 1e6 // 2**32
        rx_frac = rx_frac * 1e6 // 2**32
        tx_frac = tx_frac * 1e6 // 2**32
        
        
        t_timedelta = timedelta(microseconds=ref_frac)
        ref_time = datetime.fromtimestamp(ref_int) + t_timedelta

        t_timedelta = timedelta(microseconds=rx_frac)
        rx_time = datetime.fromtimestamp(rx_int) + t_timedelta

        t_timedelta = timedelta(microseconds=tx_frac)
        tx_time = datetime.fromtimestamp(tx_int) + t_timedelta

        orig_time = self.monotonicToDatetime(orig_time, self.start_monotonic)
        dest_time = self.monotonicToDatetime(time.monotonic(), self.start_monotonic)

        return ref_time, orig_time, rx_time, tx_time, dest_time        

In [11]:
clock_A = AbstractClock()

clock_A.correctClock()
rate = clock_A.rate
offset = clock_A.offset.total_seconds()
ts = clock_A.timestamp
t = time.monotonic()
t_corrected = ts + (t-ts)*rate + offset
print(clock_A.ntp_client.monotonicToDatetime(t_corrected, clock_A.start_monotonic))


2024-01-04 18:14:25.302164


In [12]:
class NTPclientV1():
    def __init__(self, update_rate):
        # Connection parameters
        self.host = '0.pool.ntp.org' # case of NTP online server
        #self.host = "10.42.0.1" # case of rasp
        #self.host = "192.168.0.0"
        self.port = 123
        self.read_buffer = 1024
        self.address = (self.host, self.port)
        self.epoch = 2208988800
        self.client = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)

        # Fetch parameters
        self.update_rate = update_rate # seconds
        self.fetch_time = threading.Event()

    def start(self):
        while not self.fetch_time.wait(self.update_rate):
            global counter # DEBUG
            counter = 0 # DEBUG
            t = self.getServerTime()
            
    
    def getServerTime(self):
        data = b'\x1b' + 47 * b'\0'
        orig_time = datetime.now()
        self.client.sendto(data, self.address)
        data, self.address = self.client.recvfrom(self.read_buffer)
        orig_int, orig_frac, ref_int, ref_frac, rx_int, rx_frac, tx_int, tx_frac = struct.unpack("!12I", data)[4:12] # 6:8 -> reference | 
        #ref_int -= self.epoch
        orig_int -= self.epoch
        rx_int -= self.epoch
        tx_int -= self.epoch

        # Convert the fractional part to microseconds (1 second = 2**32 fractional units)
        orig_frac = orig_frac * 1e6 // 2**32
        ref_frac = ref_frac * 1e6 // 2**32
        rx_frac = rx_frac * 1e6 // 2**32
        tx_frac = tx_frac * 1e6 // 2**32

        # Format the NTP time as a string
        ref_time = "%s.%06d" % (time.strftime("%Y-%m-%d %H:%M:%S", time.gmtime(ref_int)), ref_frac)
        orig_time = orig_time.strftime('%Y-%m-%d %H:%M:%S.%f')
        rx_time = "%s.%06d" % (time.strftime("%Y-%m-%d %H:%M:%S", time.gmtime(rx_int)), rx_frac)
        tx_time = "%s.%06d" % (time.strftime("%Y-%m-%d %H:%M:%S", time.gmtime(tx_int)), tx_frac)
        dest_time = datetime.now().strftime('%Y-%m-%d %H:%M:%S.%f')

        return ref_time, orig_time, rx_time, tx_time, dest_time

In [13]:
ABSTRACT_CLOCK_UPDATE_RATE = 0.1 # seconds 
NTP_UPDATE_RATE = 10 * ABSTRACT_CLOCK_UPDATE_RATE # seconds -> This can't be to low so that we have time to fetch the NTP time


ntp_client = NTPclient(NTP_UPDATE_RATE)


ntp_time = ntp_client.getServerTime()
print(ntp_time)

t0 = datetime.strptime(ntp_time[1], '%Y-%m-%d %H:%M:%S.%f')
t1 = datetime.strptime(ntp_time[2], '%Y-%m-%d %H:%M:%S.%f')
t2 = datetime.strptime(ntp_time[3], '%Y-%m-%d %H:%M:%S.%f')
t3 = datetime.strptime(ntp_time[4], '%Y-%m-%d %H:%M:%S.%f')


print("t0", t0)

offset = ((t1-t0) + (t2-t3)) / 2

print("Offset", offset)

#print("Send ts   ", datetime.now())
#print("NTP ts    ", ntp_client.getServerTime())
#print("Receive ts", datetime.now())



TypeError: __init__() missing 1 required positional argument: 'start_datetime'

In [ ]:
a = time.monotonic()
time.sleep(1)
b = time.monotonic()



